## Тестовая книга для проекта "Анализ лояльности пользователей Яндекс Афиши"

Проверяем подключение нужных библиотек:

In [1]:
# pandas для работы с датафреймами
import pandas as pd
# для создания соединения с удаленным сервером PostgeSQL
# и последующего SQL запроса для наполнения датафрейма данными
from sqlalchemy import create_engine

# Загружаем библиотеки для визуализации данных
import matplotlib.pyplot as plt
import seaborn as sns

# Загружаем библиотеку для расчёта коэффициента корреляции phi_k
from phik import phik_matrix

# Для работы с конфиденциальными данными
from dotenv import load_dotenv
import os

Берем параметры подключения к удаленному SQL серверу из .env файла:

In [2]:
# Загрузка файла с конфедециальными данными
load_dotenv()

# Словарь с параметрами подключения к удаленному серверу PostgeSQL
db_config = {'user': os.getenv('SQL_USER'), # имя пользователя
             'pwd': os.getenv('SQL_PWD'), # пароль
             'host': os.getenv('SQL_HOST'),
             'port': os.getenv('SQL_PORT'), # порт подключения
             'db': os.getenv('SQL_DB') # название базы данных
             }

Заполняем датафрейм при помощи SQL запроса:

In [3]:
# Строка подключения
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)
# Подключение
engine = create_engine(connection_string)
# SQL запрос к удаленному серверу для получения данных
query = '''
SELECT 
  user_id,
  device_type_canonical,
  order_id,
  created_dt_msk AS order_dt,
  created_ts_msk AS order_ts,
  currency_code,
  revenue,
  tickets_count,
  created_dt_msk::date - LAG(created_dt_msk::date) OVER (PARTITION BY user_id ORDER BY created_dt_msk) AS days_since_prev,
  event_id,
  service_name,
  event_type_main,
  region_name,
  city_name
FROM afisha.purchases
LEFT JOIN afisha.events USING(event_id)
LEFT JOIN afisha.city USING(city_id)
LEFT JOIN afisha.regions USING(region_id)
WHERE device_type_canonical IN ('mobile', 'desktop')
  AND event_type_main <> 'фильм'
ORDER BY user_id
'''
# Заполнение датафрейма
df = pd.read_sql_query(query, con=engine)

Проверяем получившийся датафрейм:

In [4]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290611 entries, 0 to 290610
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   user_id                290611 non-null  object        
 1   device_type_canonical  290611 non-null  object        
 2   order_id               290611 non-null  int64         
 3   order_dt               290611 non-null  datetime64[ns]
 4   order_ts               290611 non-null  datetime64[ns]
 5   currency_code          290611 non-null  object        
 6   revenue                290611 non-null  float64       
 7   tickets_count          290611 non-null  int64         
 8   days_since_prev        268678 non-null  float64       
 9   event_id               290611 non-null  int64         
 10  service_name           290611 non-null  object        
 11  event_type_main        290611 non-null  object        
 12  region_name            290611 non-null  obje

,user_id,device_type_canonical,order_id,order_dt,order_ts,currency_code,revenue,tickets_count,days_since_prev,event_id,service_name,event_type_main,region_name,city_name
0,0002849b70a3ce2,mobile,4359165,2024-08-20,2024-08-20 16:08:03,rub,1521.94,4,NaN,169230,Край билетов,театр,Каменевский регион,Глиногорск
1,0005ca5e93f2cf4,mobile,7965605,2024-07-23,2024-07-23 18:36:24,rub,289.45,2,NaN,237325,Мой билет,выставки,Каменевский регион,Глиногорск
2,0005ca5e93f2cf4,mobile,7292370,2024-10-06,2024-10-06 13:56:02,rub,1258.57,4,75.0,578454,За билетом!,другое,Каменевский регион,Глиногорск
3,000898990054619,mobile,1139875,2024-07-13,2024-07-13 19:40:48,rub,8.49,2,NaN,387271,Лови билет!,другое,Североярская область,Озёрск
4,000898990054619,mobile,972400,2024-10-04,2024-10-04 22:33:15,rub,1390.41,3,83.0,509453,Билеты без проблем,стендап,Озернинский край,Родниковецк
